In [38]:
import numpy as np
from tabulate import tabulate
import math

Заданные функции

In [39]:
def f(x, y, z):
    return (x + (1/x) - (x + 2)*z + y) / (x*(x + 1))
def g(x, y, z):
    return z
def p(x):
    return (x + 2) / (x * (x + 1))
def q(x):
    return -1 * (1 / (x * (x + 1)))
def f2(x):
    return (x + (1/x)) / (x * (x + 1))
def yf(x):
    return x + 7/2 + (1/x) + ((x/2) + 1)*math.log(abs(x))

Методы Рунге-Кутты стрельбы

In [40]:
def runge_kutty_method(y0, z0, xl, xr, h):
    """
    Решает задачу Коши для ОДУ 2-го порядка методом Рунге-Кутты 4-го порядка.
    
    Параметры:
    y0 - начальное значение y.
    z0 - начальное значение z.
    xl - начальная точка интервала интегрирования.
    xr - конечная точка интервала интегрирования.
    h - шаг интегрирования.
    
    Возвращает:
    Списки значений x, y, z на интервале интегрирования.
    """
    x = [i for i in np.arange(xl, xr + h, h)]
    y = [y0]
    z = [z0]
    dz = []
    dy = []
    Kz = []
    Ky = []
    
    # Основной цикл метода Рунге-Кутты
    for i in range(len(x) - 1):
        L1 = h * f(x[i], y[i], z[i])
        K1 = h * g(x[i], y[i], z[i])
        L2 = h * f(x[i] + 0.5 * h, y[i] + 0.5 * K1, z[i] + 0.5 * L1)
        K2 = h * g(x[i] + 0.5 * h, y[i] + 0.5 * K1, z[i] + 0.5 * L1)
        L3 = h * f(x[i] + 0.5 * h, y[i] + 0.5 * K2, z[i] + 0.5 * L2)
        K3 = h * g(x[i] + 0.5 * h, y[i] + 0.5 * K2, z[i] + 0.5 * L2)
        L4 = h * f(x[i] + h, y[i] + K3, z[i] + L3)
        K4 = h * g(x[i] + h, y[i] + K3, z[i] + L3)
        Ky.append([K1, K2, K3, K4])
        Kz.append([L1, L2, L3, L4])
        
        # Вычисление приращений dy и dz по формуле Рунге-Кутты
        dy.append((K1 + 2 * K2 + 2 * K3 + K4) / 6)
        dz.append((L1 + 2 * L2 + 2 * L3 + L4) / 6)
        
        y.append(y[i] + dy[i])
        z.append(z[i] + dz[i])
    
    return x, y, z

In [41]:
def shooting_method(x0, y0, x1, y1, h):
    # Задание начального допущения и начальной точности
    eps = 0.0001
    tt = [1.0, 0.8]
    
    # Решение уравнения Рунге-Кутты для двух начальных приближений
    xn1, yn1, zn1 = runge_kutty_method(y0, tt[0], x0, x1, h)
    xn2, yn2, zn2 = runge_kutty_method(y0, tt[1], x0, x1, h)
    
    y = [yn1[-1], yn2[-1]]
    Phi = [abs(yn1[-1] - y1), abs(yn2[-1] - y1)]
    i = 1
    
    # Цикл для нахождения корректного значения начального условия методом секущих
    while abs(Phi[i]) > eps:
        i += 1
        tt.append(tt[i - 1] - ((tt[i - 1] - tt[i - 2]) / (Phi[i - 1] - Phi[i - 2])) * Phi[i - 1])
        xni, yni, zni = runge_kutty_method(y0, tt[i], x0, x1, h)
        Phi.append(abs(yni[-1] - y1))
        y.append(yni[-1])
        
    return tt, y, Phi, xni, yni

Конечно-разностный метод

In [42]:
def progon_method(a, b):
    n = len(a)
    P = np.zeros(n)
    Q = np.zeros(n)
    
    # Вычисление прогоночных коэффициентов P и Q
    P[0] = -a[0][1] / a[0][0]
    Q[0] = b[0] / a[0][0]
    for i in range(1, n):
        denominator = a[i][i] + a[i][i - 1] * P[i - 1]
        if i == n - 1:
            P[i] = 0
        else:
            P[i] = -a[i][i + 1] / denominator
        Q[i] = (b[i] - a[i][i - 1] * Q[i - 1]) / denominator
        
    x = np.zeros(n)
    x[n - 1] = Q[n - 1]
    
    # Обратный ход для нахождения решения x
    for i in range(n - 2, -1, -1):
        x[i] = P[i] * x[i + 1] + Q[i]
        
    return x


In [43]:
def finite_difference(x0, x1, y0, y1, h):
    # Задание сетки x
    x = [i for i in np.arange(x0, x1 + h, h)]
    A = []
    b = []
    
    # Заполнение матрицы коэффициентов A и вектора правых частей b
    for i in range(1, len(x)):
        if i == 1:
            a = []
            a.append(-2 + h * h * q(x[i]))
            a.append(1 + p(x[i]) * h / 2)
            for j in range(2, len(x)):
                a.append(0)
            A.append(a)
            b.append(h * h * f2(x[i]) - (1 - p(x[i]) * h / 2) * y0)
        elif i == len(x) - 1:
            a = []
            for j in range(1, len(x) - 2):
                a.append(0)
            a.append(1 - p(x[i]) * h / 2)
            a.append(-2 + h * h * q(x[i]))
            A.append(a)
            b.append(h * h * f2(x[i]) - (1 + p(x[i]) * h / 2) * y1)
        else:
            a = []
            for j in range(1, i - 1):
                a.append(0)
            a.append(1 - p(x[i]) * h / 2)
            a.append(-2 + h * h * q(x[i]))
            a.append(1 + p(x[i]) * h / 2)
            for j in range(i + 2, len(x)):
                a.append(0)
            A.append(a)
            b.append(h * h * f2(x[i]))
            
    # Решение системы линейных уравнений методом прогонки
    y = progon_method(A, b)
    
    return x, y

Входные данные

In [44]:
x0 = 1
y0 = 3/2
x1 = 2
y1 = 13 + 4 * np.log(2)
h = (x1 - x0) / 5

Вывод метода стрельбы

In [45]:
tt, y, Phi, xni, yni = shooting_method(x0, y0, x1, y1, h)
table = [["k", "этта", "y(b, y0, tt)", "Phi"]]
table1 = [["k", "xk", "yk", "yf"]]
for i in range(len(Phi)):
    table.append([i, tt[i],y[i], Phi[i]])
for i in range(len(xni)):
    table1.append([i, xni[i], yni[i], yf(xni[i])])
print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))
print(tabulate(table1, headers='firstrow', tablefmt='fancy_grid'))
tt, y, Phi, xni, yni2 = shooting_method(x0, y0, x1, y1, 2*h)
tt, y, Phi, xni, yni1 = shooting_method(x0, y0, x1, y1, h)

╒═════╤═════════╤════════════════╤══════════════╕
│   k │    этта │   y(b, y0, tt) │          Phi │
╞═════╪═════════╪════════════════╪══════════════╡
│   0 │  1      │        2.80123 │ 12.9714      │
├─────┼─────────┼────────────────┼──────────────┤
│   1 │  0.8    │        2.65942 │ 13.1132      │
├─────┼─────────┼────────────────┼──────────────┤
│   2 │ 19.2943 │       15.7726  │  7.10543e-15 │
╘═════╧═════════╧════════════════╧══════════════╛
╒═════╤══════╤══════════╤═════════╕
│   k │   xk │       yk │      yf │
╞═════╪══════╪══════════╪═════════╡
│   0 │  1   │  1.5     │ 5.5     │
├─────┼──────┼──────────┼─────────┤
│   1 │  1.2 │  4.90507 │ 5.82505 │
├─────┼──────┼──────────┼─────────┤
│   2 │  1.4 │  7.66365 │ 6.18629 │
├─────┼──────┼──────────┼─────────┤
│   3 │  1.6 │ 10.0189  │ 6.57101 │
├─────┼──────┼──────────┼─────────┤
│   4 │  1.8 │ 12.1062  │ 6.97235 │
├─────┼──────┼──────────┼─────────┤
│   5 │  2   │ 14.0068  │ 7.38629 │
├─────┼──────┼──────────┼─────────┤
│   6 │  2

Вывод конечно-разностного метода

In [46]:
x, y = finite_difference(x0, x1, y0, y1, h)
table = [["k", "xk", "yk", "yf"]]
table.append([0, x0, y0, yf(x[0])])

for i in range(0, len(y)):
    table.append([i+1, x[i+1], y[i], yf(x[i+1])])
print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))

x, yk1 = finite_difference(x0, x1, y0, y1, h/2)
x, yk2 = finite_difference(x0, x1, y0, y1, h)

╒═════╤══════╤══════════╤═════════╕
│   k │   xk │       yk │      yf │
╞═════╪══════╪══════════╪═════════╡
│   0 │  1   │  1.5     │ 5.5     │
├─────┼──────┼──────────┼─────────┤
│   1 │  1.2 │  4.52877 │ 5.82505 │
├─────┼──────┼──────────┼─────────┤
│   2 │  1.4 │  6.99134 │ 6.18629 │
├─────┼──────┼──────────┼─────────┤
│   3 │  1.6 │  9.09978 │ 6.57101 │
├─────┼──────┼──────────┼─────────┤
│   4 │  1.8 │ 10.9726  │ 6.97235 │
├─────┼──────┼──────────┼─────────┤
│   5 │  2   │ 12.6811  │ 7.38629 │
├─────┼──────┼──────────┼─────────┤
│   6 │  2.2 │ 14.271   │ 7.81031 │
╘═════╧══════╧══════════╧═════════╛


Оценка погрешности методом Рунге-Ромберга

In [47]:
def runge_rom(y1, y2, r, p):
    k = -10
    for i in range(min(len(y1), len(y2))):
        k = max(k, abs((y1[i] - y2[i]) / (r**p - 1))) 
    return k

table = [["Стрельбы", "Конечно-Разностный"]]
table.append([runge_rom(yni1, yni2, 2, 4), runge_rom(yk1, yk2, 2, 1)])
print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))

╒════════════╤══════════════════════╕
│   Стрельбы │   Конечно-Разностный │
╞════════════╪══════════════════════╡
│    0.38358 │              3.69267 │
╘════════════╧══════════════════════╛
